# Лабораторна робота №3
## Варіант №3
Мета роботи:
* Дослідити методи розрахунку ваг альтернатив за критерієм на основі нечітких експертних оцінок парних порівнянь на основі методу нижньої і верхньої апроксимацій LUAM

In [1]:
%pylab inline
import pandas as pd
import itertools
import scipy
import scipy.optimize
from IPython.display import Latex
from IPython.core.display import display, HTML

Populating the interactive namespace from numpy and matplotlib


## Нижня модель LUAM
<img src="./lower_model.png"/>
## Верхня модель LUAM
<img src="./upper_model.png"/>

In [2]:
# Get from lab2_1
def cr_metric(lamb, dim):
    """
    Eats matrix of pairwise comparisons m and produces CR metric of consistensy.
    """
    # calculate ci metric
    assert lamb >= dim, "Greatest eigenvalue must be greater than matrix dimension."
    ci = (lamb - dim)
    if dim > 1:
        ci /= float(dim - 1)
    # fill the MRCI table
    mrci = [0, 0, 0, 0.52, 0.89, 1.11, 1.25, 1.35, 1.40, 1.45, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59] 
    cr = 0 if ci == 0 else ci / mrci[dim]
    return cr

def em_cr(mat):
    """
    Eats matrix of pairwise comparisons m and produces pair (local weights, CR metric).
    """
    assert mat.shape[0] == mat.shape[1]
    values, vectors = linalg.eig(mat)
    max_index = np.argmax(abs(values))
    v = vectors[:, max_index]
    lamb = values[max_index]
    assert (v.imag < 1e-6).all(), "eigenvector has complex coordinates: %s" % v.imag
    assert lamb.imag < 1e-6, "eigenvalue has complex coordinates: %s" % lamb.imag
    v = v.real
    lamb = lamb.real
    if (v < 0).all():
        v = - v
    assert (v >= 0).all(), "eigenvector has negative coordinates: %s" % v
    return v / linalg.norm(v, ord=1), cr_metric(lamb, mat.shape[0])

A = np.array([
    [[1, 1], [3, 4], [6, 6], [6, 7]],
    [[1/4, 1/3], [1, 1], [3, 4], [3, 4]],
    [[1/6, 1/6], [1/4, 1/3], [1, 1], [3, 4]],
    [[1/7,1/6], [1/4, 1/3], [1/4, 1/3], [1, 1]]
])

A = np.array([
    [[1, 1], [1, 3], [3, 5], [5, 7], [5, 9]],
    [[1/3, 1], [1, 1], [1, 4], [1, 5], [1, 4]],
    [[1/5, 1/3], [1/4, 1], [1, 1], [1/5, 5], [2, 4]],
    [[1/7, 1/5], [1/5, 1], [1/5, 5], [1, 1], [1, 2]],
    [[1/9, 1/5], [1/4, 1], [1/4, 1/2], [1/2, 1], [1, 1]]
])
AL = A[:, :, 0]
AU = A[:, :, 1]

def fill_lower(r, a, k, i, j):
    r[k, j + a.shape[1]] = a[i, j]
    r[k, i] = -1

def fill_upper(r, a, k, i, j):
    r[k,  i + a.shape[0]] = 1
    r[k,  j] = -a[i, j]
        
def create_matrix(a, f):
    " w_1^L, ... w_n^L, w_1^U, ..., w_n^[l]"
    n, _ =  a.shape
    r = np.zeros((n * n, 2 * n))
    for k, p in enumerate(itertools.product(range(n), range(n))):
        if (p[0] != p[1]):
            f(r, a, k, p[0], p[1])
    return r

def third_lower(n):
    r = np.zeros((n, 2 * n))
    for i in range(n):
        r[i, i] = -1
        for j in range(n):
            if i != j:
                r[i, j + n] = -1
    return r

def fourth_lower(n):
    r = np.zeros((n, 2 * n))
    for i in range(n):
        r[i, i + n] = 1
        for j in range(n):
            if i != j:
                r[i, j] = 1
    return r

def fiveth_lower(n):
    r = np.zeros((n, 2 * n))
    for i in range(n):
        r[i, i] = 1
        r[i, i + n] = -1
    return r
        
def solve(l, u, sz=1):
    m, n = l.shape
    s = 2 * n * n
    assert(m == n)
    assert(l.shape == u.shape)
    a1 = sz * create_matrix(l, fill_lower)
    a2 = sz * create_matrix(u, fill_upper)
    a3 = third_lower(n)
    a4 = fourth_lower(n)
    a5 = fiveth_lower(n)
    a_components = [a1, a2, a3, a4, a5]
    a = np.concatenate(a_components)
    b = np.zeros(s + 3 * n)
    b[s:s + n] = -1
    b[s + n: s + 2 * n] = 1
    c = np.zeros(n + n)
    c[0:n] = sz
    c[n:] = -sz
    data = [c, a, b]
    result = scipy.optimize.linprog(
        *data,
        options=dict(disp=True),
        bounds=[(0.0001, np.inf) for i in range(n + n)]
    )
    return data, result

def get_ranges(solution):
    x = solution.x
    n = len(solution.x) // 2
    return list(zip(x[0:n], x[n:]))

def fuzzy_advantage(w_low, w_mid, w_up, j_s=0.2, j_e=0.2):
    mm = np.meshgrid(w_mid, w_mid)
    lu = np.meshgrid(w_low, w_up)
    vss = np.zeros_like(mm[0])
    vss[mm[1] >= mm[0]] = 1
    mask_mixed = (mm[1] < mm[0]) * (lu[1] >= lu[0])

    vals_mixed = (lu[1] - lu[0])[mask_mixed] / (lu[1] - mm[1] + mm[0] - lu[0])[mask_mixed]
    vss[mask_mixed] = vals_mixed#[mask_mixed]
    print('Matrix V:\n',vss)
    vs = np.clip(vss - vss.T, 0, 1)
    ve = np.minimum(vss, vss.T)
    order_g = vs >= j_s
    order_eq = ve >= j_e
    print('">" relation:\n', order_g)
    print('"~" relation:\n', order_eq)
    return rank_by_g_order(order_g)

def rank_by_g_order(order):
    marked_indices = np.arange(order.shape[0])
    while (marked_indices.size > 0):
        order_slice = order[np.ix_(marked_indices, marked_indices)]
        greatest = np.where(np.sum(order_slice, axis=0) == 0)[0]
        yield marked_indices[greatest]
        marked_indices = np.delete(marked_indices, greatest)


In [3]:
def prepare_report(_, solution, n):
    ranges = get_ranges(solution)
    sorted_ranges = sorted(enumerate(ranges), key=lambda x: -(sum(x[1])))
    pattern = "$[w%d^{L}_{%d}, w%d^{U}_{%d}] = [%f, %f]$"
    sorted_ranges_latex = [pattern % (n, i + 1, n, i + 1, rng[0], rng[1]) for i, rng in sorted_ranges]
    J = "$J^{*} = %f$" % np.abs(solution.fun)
    weights = '<br/>'.join(sorted_ranges_latex)
    ordering = "%s" % ("$\succeq$".join([str(i + 1) for i, _ in sorted_ranges]))
    return """
        %s<br/>
        %s<br/>
        %s<br/>
    """ % (J, weights, ordering)

In [4]:
solution1 = solve(AL, AU, sz=1)
r1 = prepare_report(*solution1, 1)
solution2 = solve(AL, AU, sz=-1)
r2 = prepare_report(*solution2, 2)

Optimization terminated successfully.
         Current function value: -0.223535   
         Iterations: 45
Optimization terminated successfully.
         Current function value: 0.618182    
         Iterations: 23


# Розв'язок нижньої моделі

In [5]:
HTML(r1)

# Розв'язок верхньої моделі

In [6]:
HTML(r2)

In [7]:
def rank(solution):
    advantages = np.array(get_ranges(solution))
    l = advantages[:, 0].ravel()
    u = advantages[:, 1].ravel()
    #print(l)
    #print(u)
    return fuzzy_advantage(l, (l + u) / 2, u)

In [8]:
list(rank(solution1[1]))

Matrix V:
 [[ 1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  1.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  1.]]
">" relation:
 [[False  True  True  True  True]
 [False False  True  True  True]
 [False False False  True  True]
 [False False False False  True]
 [False False False False False]]
"~" relation:
 [[ True False False False False]
 [False  True False False False]
 [False False  True False False]
 [False False False  True False]
 [False False False False  True]]


[array([0]), array([1]), array([2]), array([3]), array([4])]

In [9]:
ranks = list(list(r + 1) for r in rank(solution2[1]))

Matrix V:
 [[  1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
    1.00000000e+00]
 [  0.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
    1.00000000e+00]
 [  0.00000000e+00   2.94117647e-01   1.00000000e+00   1.00000000e+00
    1.00000000e+00]
 [  0.00000000e+00   2.18079523e-16   8.57142857e-01   1.00000000e+00
    9.52380952e-01]
 [  0.00000000e+00   0.00000000e+00   8.88888889e-01   1.00000000e+00
    1.00000000e+00]]
">" relation:
 [[False  True  True  True  True]
 [False False  True  True  True]
 [False False False False False]
 [False False False False False]
 [False False False False False]]
"~" relation:
 [[ True False False False False]
 [False  True  True False False]
 [False  True  True  True  True]
 [False False  True  True  True]
 [False False  True  True  True]]


In [10]:
ranks

[[1], [2], [3, 4, 5]]

## EM_CR

In [11]:
em_cr(np.sqrt(AL * AU))

(array([ 0.47707239,  0.21985661,  0.13063933,  0.1000566 ,  0.07237507]),
 0.026402957695905803)

In [12]:
weight, CR_metric = em_cr(np.sqrt(AL * AU))
sorted_a = sorted(range(1, len(weight)+1), key= lambda i: weight[i-1], reverse=True)
print('CR:\n', CR_metric)
print('Weight:\n', weight)
print('Sorted:\n', sorted_a)

CR:
 0.0264029576959
Weight:
 [ 0.47707239  0.21985661  0.13063933  0.1000566   0.07237507]
Sorted:
 [1, 2, 3, 4, 5]


## Висновки
В ході виконання лабораторної роботи, ми скористалися методом LUAM для отримання інтервальних ваг на базі інтервальної матриці парних порівнянь. Отримали ранжування, за яким розташовані у наступному порядку: $1, 2, 3, 4, 5$.
Отримане ранжування за методом LUAM співпадає з ранжуванням, отримане методом em для геометричного середнього для граничних значень інтервалів матриці парних порівнянь.